#RBM

In [20]:
import os
import numpy as np
import pylab as pl
%pylab inline

from sklearn.metrics import mean_squared_error as mse 
from scipy import misc
from scipy.special import expit

Populating the interactive namespace from numpy and matplotlib


In [49]:
DATA_PATH = "../DataSets/big_alphabet_29x29/"

IMAGE_SIZE = (29, 29)
IMAGE_LENGTH = 29*29

In [39]:
class RBM:
    def __init__(self, visible_size, hidden_size, n_steps=1000, k=1, learning_rate=0.1, verbose=True):
        if k < 1:
            raise ValueError("k must be integer greater than 0")
        
        self.visible_size = visible_size
        self.hidden_size = hidden_size
        self.n_steps = n_steps
        self.k = k
        self.learning_rate = learning_rate
        
        self.w = np.random.normal(0, 0.01, (self.visible_size, self.hidden_size))
        self.a = np.random.normal(0, 0.01, self.visible_size)
        self.b = np.random.normal(0, 0.01, self.hidden_size)
        self.verbose = verbose
        
    def fit(self, X):
        errors = np.zeros(self.n_steps)
        for i in range(self.n_steps):
            #CD-k step
            v, h = X[np.random.choice(X.shape[0]), ::], None
            v_start, h_start = None, None
            for j in range(self.k):
                h = expit(np.dot(v, self.w) + self.b)
                if j == 0:
                    v_start, h_start = v, h
                v = expit(np.dot(self.w, h).T + self.a)
            
            errors[i] = mse(v_start, self.predict(v_start))
            if self.verbose:
                print "Step %d, mse reconstruction %s" % (i, errors[i])
                
            #update step
            gradW = (np.dot(v_start, h_start.T) - np.dot(v, h.T))
            gradA = self.learning_rate * (v_start - v)
            gradB = self.learning_rate * (h_start - h)
            
            self.w += self.learning_rate * gradW
            self.a += self.learning_rate * gradA
            self.b += self.learning_rate * gradB
            
        return errors
            
    def predict(self, sample):
        h = expit(np.dot(sample, self.w) + self.b)
        v = expit(np.dot(self.w, h).T + self.a)
        return v

In [50]:
data = list()

for pt in os.listdir(DATA_PATH):
    im = misc.imread(os.path.join(DATA_PATH, pt), flatten=True).flatten() / 255.0                   
    data.append(im)
    
data = np.array(data)

In [43]:
rbm = RBM(visible_size=DATA_LENGTH, hidden_size=DATA_LENGTH, k=5, n_steps=1000)

In [44]:
errors = rbm.fit(data)

Step 0, mse reconstruction 0.245771742388
Step 1, mse reconstruction 0.122473246136
Step 2, mse reconstruction 0.0868014268728
Step 3, mse reconstruction 0.249392127176
Step 4, mse reconstruction 0.248384921539
Step 5, mse reconstruction 0.247915131606
Step 6, mse reconstruction 0.24642719294
Step 7, mse reconstruction 0.245312326732
Step 8, mse reconstruction 0.24448135719
Step 9, mse reconstruction 0.244355473709
Step 10, mse reconstruction 0.242670604922
Step 11, mse reconstruction 0.24227959401
Step 12, mse reconstruction 0.241155272248
Step 13, mse reconstruction 0.239951624361
Step 14, mse reconstruction 0.23936454265
Step 15, mse reconstruction 0.238512977254
Step 16, mse reconstruction 0.237278299724
Step 17, mse reconstruction 0.23609635045
Step 18, mse reconstruction 0.235186277318
Step 19, mse reconstruction 0.234948848355
Step 20, mse reconstruction 0.233721130441
Step 21, mse reconstruction 0.232823156278
Step 22, mse reconstruction 0.231703389527
Step 23, mse reconstructi

In [63]:
sample = data[np.random.choice(data.shape[0]), ::]
prediction = rbm.predict(sample)

f, axarr = pl.subplots((1, 2), sharex=True)
f.set_size_inches(3, 6)
axarr[0].imshow(sample.reshape(DATA_SIZE), cmap="gray")
axarr[0].set_title("original")
axarr[0].axis('off')
axarr[1].imshow(prediction.reshape(DATA_SIZE), cmap="gray")
axarr[1].set_title("prediction")
axarr[1].axis('off')
pl.show()

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'